# Mouse RNAseq analysis with DESeq2
we use an R kernel here.

Install DESeq2 `BiocManager::install("DESeq2")` (didn't update any of the additional packages when asked)   
Install tidyverse `install.packages('tidyverse')`   
Install apeglm `BiocManager::install("apeglm")`   


In [1]:
library(DESeq2)
library(tidyverse)

Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: 'BiocGenerics'


The following objects are masked from 'package:parallel':

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs


The following objects are masked from 'package:base':

    Filter, Find, Map, Position, Reduce, anyDuplicated, append,
    as.data.frame, basename, cbind, colnames, dirname, do.call,
    duplicated, eval, evalq, get, grep, grepl, intersect, is.unsorted,
    lapply, mapply, match, mget, order, paste, pmax, pmax.int, pmin,
    pmin.int, rank, rbind, rownames, sapply, setdiff, sort, table,
    tapply, union, unique, unsplit, which.max, which.min



Attaching package: 'S4Vectors'


The follow

In [2]:
DATA_DIR = "/lustre/groups/ml01/workspace/louis.kuemmerle/projects/A1/raw_data/230210_rnaseq_mouse/"
RESULTS_DIR = "/lustre/groups/ml01/workspace/louis.kuemmerle/projects/A1/results/230210_rnaseq_mouse/"

#"counts/nfc-rnaseq-3.9/report1090/output/results/star_salmon/deseq2_qc/deseq2.dds.RData"
#"counts/nfc-rnaseq-3.9/report1090/output/results/star_salmon/salmon.merged.gene_counts.rds"
#"counts/P23010_sample_sheet.csv"
#"fastqs/Project_588_lims.csv"

### Load counts and obs

In [3]:
cts <- as.matrix(read.csv(paste0(DATA_DIR, "counts/nfc-rnaseq-3.9/report1090/output/results/star_salmon/salmon.merged.gene_counts.tsv"),
                          sep="\t",row.names="gene_id"))
cts<-cts[,-1]
mode(cts) <- "integer"
sampleinfo <- read_csv(paste0(DATA_DIR, "Project_588_lims_clean.csv"),col_types="cccc")
rownames(sampleinfo) <- sampleinfo$Sample_Name
#coldata <- read.csv(paste0(DATA_DIR, "Project_588_lims_clean.csv"))#, row.names=1)

Warning message:
"Setting row names on a tibble is deprecated."


### Functions for DE tests

In [7]:
DE_test_conditions <- function(cts_mat, sinfo, region) {
    #"""
    #cts_mat: cts, potentially subsetted to samples of interest
    #sinfo: sampleinfo, potentially subsetted to samples of interest
    #region: only for naming of the created csv file. E.g. region='all' 
    #
    #"""
    
    keys <- c("region", "condition", "Naive", "Sham", "MCAo")
    coldata <- sinfo[,keys]
    for (key in keys) {
        coldata[[key]] <- factor(coldata[[key]])
    }

    # Tests N vs M, S vs M, N vs S
    dds <- DESeqDataSetFromMatrix(countData = cts_mat, colData = coldata, design = ~ condition)
    keep <- rowSums(counts(dds)) >= 10
    dds <- dds[keep,]

    dds$condition <- relevel(dds$condition, ref = "MCAo")
    dds_ <- DESeq(dds)
    res <- results(dds_, name="condition_Naive_vs_MCAo")
    res <- res[!is.na(res$padj),]
    write.csv(res, paste0(RESULTS_DIR, "DE_tests/", "region_", region, "_condition_Naive_vs_MCAo.csv"))
    res <- results(dds_, name="condition_Sham_vs_MCAo")
    res <- res[!is.na(res$padj),]
    write.csv(res, paste0(RESULTS_DIR, "DE_tests/", "region_", region, "_condition_Sham_vs_MCAo.csv"))

    dds$condition <- relevel(dds$condition, ref = "Sham")
    dds_ <- DESeq(dds)
    res <- results(dds_, name="condition_Naive_vs_Sham")
    res <- res[!is.na(res$padj),]
    write.csv(res, paste0(RESULTS_DIR, "DE_tests/", "region_", region, "_condition_Naive_vs_Sham.csv"))

    # Tests N vs S+M, S vs N+M, M vs N+S
    for (cond in c("Naive", "Sham", "MCAo")) {
        dds <- DESeqDataSetFromMatrix(countData = cts_mat, colData = coldata, design = as.formula(sprintf(" ~ `%s`", cond))) #design = ~ Naive/Sham/MCAo
        keep <- rowSums(counts(dds)) >= 10        
        dds <- dds[keep,]
        
        dds[[cond]] <- relevel(dds[[cond]], ref = "other")
        dds_ <- DESeq(dds)
        res <- results(dds_, name=paste0(cond,"_",cond,"_vs_other"))
        res <- res[!is.na(res$padj),]
        write.csv(res, paste0(RESULTS_DIR, "DE_tests/", "region_", region, "_condition_", cond, "_vs_other.csv"))
    }

}

DE_test_regions <- function(cts_mat, sinfo, cond) {
    #"""
    #cts_mat: cts, potentially subsetted to samples of interest
    #sinfo: sampleinfo, potentially subsetted to samples of interest
    #cond: only for naming of the created csv file
    #
    #"""   
    
    bones <- c("Calvaria", "Vertebra", "Scapula", "Femur", "Humerus", "Pelvis")
    coldata <- sinfo[,bones]
    for (r in bones) {
        coldata[[r]] <- factor(coldata[[r]])
    }
    
    # Tests each region vs all others
    for (r in bones) {
        dds <- DESeqDataSetFromMatrix(countData = cts_mat, colData = coldata, design = as.formula(sprintf(" ~ `%s`", r))) #design = ~ Naive/Sham/MCAo
        keep <- rowSums(counts(dds)) >= 10
        dds <- dds[keep,]
        
        dds[[r]] <- relevel(dds[[r]], ref = "other")
        dds_ <- DESeq(dds)
        res <- results(dds_, name=paste0(r,"_",r,"_vs_other"))
        res <- res[!is.na(res$padj),]
        write.csv(res, paste0(RESULTS_DIR, "DE_tests/", "condition_", cond, "_region_", r, "_vs_other.csv"))
    }
}

# DE Tests

In [10]:
# Condition tests including all regions
DE_test_conditions(cts, sampleinfo, "all")

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 840 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 840 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 925 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- or

In [9]:
# Condition tests per regions
bones <- c("Calvaria", "Vertebra", "Scapula", "Femur", "Humerus", "Pelvis")
for (r in bones) {
    keep_samples <- (sampleinfo$region == r)
    cts_subset <- cts[,keep_samples]
    sinfo <- sampleinfo[keep_samples,]
    
    DE_test_conditions(cts_subset, sinfo, r)
}

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 269 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 406 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in co

In [8]:
# Region tests per condition
conditions <- c("Naive", "Sham", "MCAo")
for (c in conditions) {
    keep_samples <- (sampleinfo$condition == c)
    cts_subset <- cts[,keep_samples]
    sinfo <- sampleinfo[keep_samples,]
    
    DE_test_regions(cts_subset, sinfo, c)
}

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 161 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

-- note: fitType='parametric', but the dispersion trend was not well captured by the
   function: y = a/x + b, and a local regression fit was automatically substituted.
   specify fitType='local' or 'mean' to avoid this message next time.

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 419 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing

estimating size factors

estim